In [ ]:
# 필수 라이브러리
import pandas as pd
import numpy as np

# 전체 데이터셋 읽어들이기
raw_df = pd.read_csv('./data/continuous_factory_process.csv', index_col="time_stamp")

# 2. Setpoint가 포함된 column 제거
# 지시사항에 따라 'Setpoint'가 포함된 column을 제거
# 'Setpoint'는 모델 학습에 불필요한 정보이므로 input feature에서 제거해야 함
df = raw_df.drop(columns=[col for col in raw_df.columns if 'Setpoint' in col])

# 3. Stage 1, Stage 2의 output feature name을 정의
# 공정의 첫 번째 단계(Stage 1)와 두 번째 단계(Stage 2)에서 측정된 제품의 품질 데이터를 output으로 정의해야 함
# 각 Stage에는 15개의 output feature가 있음
stage1_output = [col for col in df.columns if 'Stage1.Output' in col]
stage2_output = [col for col in df.columns if 'Stage2.Output' in col]
print(len(stage1_output), len(stage2_output))

# 4. Stage 1, Stage 2의 input feature name 정의
stage1_input = [col for col in df.columns if col not in stage1_output]
stage2_input = [col for col in df.columns if col not in stage2_output]

# 5. Stage 1, Stage 2 데이터셋 구성
stage1_input_df = df[stage1_input]
stage1_output_df = df[stage1_output]
stage2_input_df = df[stage2_input]
stage2_output_df = df[stage2_output]

# 6. test set(X_test) 데이터 읽어들이기
# 학습한 모델로 테스트할 데이터(submission_data.csv)를 읽어옴
# 이 데이터는 실제로 예측할 데이터(X_test)에 해당함
submission_df = pd.read_csv('./data/submission_data.csv', index_col="time_stamp")

# 7. Setpoint가 포함된 column 제거
# 학습 데이터와 동일하게 Setpoint가 포함된 column을 제거하여 모델 학습에 사용한 feature들과 일치시킴
submission_df = submission_df.drop(columns=[col for col in submission_df.columns if 'Setpoint' in col])

# 8. Stage 1 및 Stage 2의 input 데이터 구성
# ※ 주의사항 : 학습 데이터에서의 input feature와 동일하게 구성해야 합 ※
stage1_submission_input_df = submission_df[stage1_input]
stage2_submission_input_df = submission_df[stage2_input]

# 9. NaN 값 처리
# NaN 값을 각 feature의 평균값으로 대체
stage1_input_df = stage1_input_df.fillna(stage1_input_df.mean())
stage2_input_df = stage2_input_df.fillna(stage2_input_df.mean())
stage1_submission_input_df = stage1_submission_input_df.fillna(stage1_submission_input_df.mean())
stage2_submission_input_df = stage2_submission_input_df.fillna(stage2_submission_input_df.mean())

# 10. 모델 학습을 위한 라이브러리 불러오기
# 여기에서는 임의로 LinearRegression 모델을 사용하지만, 상황에 따라 다른 모델 선정 가능
from sklearn.linear_model import LinearRegression

# 출력 데이터가 2D로 되어 있는지 확인 후 변환 (필요 시)
if len(stage1_output_df.shape) == 1:
    stage1_output_df = stage1_output_df.values.reshape(-1, 15)

if len(stage2_output_df.shape) == 1:
    stage2_output_df = stage2_output_df.values.reshape(-1, 15)

# 10. 모델 학습
stage1_model = LinearRegression()
stage1_model.fit(stage1_input_df, stage1_output_df)

stage2_model = LinearRegression()
stage2_model.fit(stage2_input_df, stage2_output_df)

from sklearn.impute import SimpleImputer

# NaN 값 평균으로 대체
imputer = SimpleImputer(strategy='mean')

# Stage 1 input 데이터 결측값 대체
stage1_input_df = pd.DataFrame(imputer.fit_transform(stage1_input_df), columns=stage1_input_df.columns)
stage1_submission_input_df = pd.DataFrame(imputer.transform(stage1_submission_input_df), columns=stage1_submission_input_df.columns)

# Stage 2 input 데이터 결측값 대체
stage2_input_df = pd.DataFrame(imputer.fit_transform(stage2_input_df), columns=stage2_input_df.columns)
stage2_submission_input_df = pd.DataFrame(imputer.transform(stage2_submission_input_df), columns=stage2_submission_input_df.columns)

# 결측값 처리 후 다시 학습 및 예측
stage1_model = LinearRegression()
stage1_model.fit(stage1_input_df, stage1_output_df)

stage2_model = LinearRegression()
stage2_model.fit(stage2_input_df, stage2_output_df)

# 예측 수행
stage1_predictions = stage1_model.predict(stage1_submission_input_df)
stage2_predictions = stage2_model.predict(stage2_submission_input_df)

In [ ]:
# 필수 라이브러리
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer

# 1. 전체 데이터셋 읽어들이기
raw_df = pd.read_csv('./data/continuous_factory_process.csv', index_col="time_stamp")

# 2. Setpoint가 포함된 column 제거
df = raw_df.drop(columns=[col for col in raw_df.columns if 'Setpoint' in col])

# 3. Stage 1, Stage 2의 output feature name을 정의
stage1_output = [col for col in df.columns if 'Stage1.Output' in col]
stage2_output = [col for col in df.columns if 'Stage2.Output' in col]
print(len(stage1_output), len(stage2_output))

# 4. Stage 1, Stage 2의 input feature name 정의
stage1_input = [col for col in df.columns if col not in stage1_output]
stage2_input = [col for col in df.columns if col not in stage2_output]

# 5. Stage 1, Stage 2 데이터셋 구성
stage1_input_df = df[stage1_input]
stage1_output_df = df[stage1_output]
stage2_input_df = df[stage2_input]
stage2_output_df = df[stage2_output]

# 6. test set(X_test) 데이터 읽어들이기
submission_df = pd.read_csv('./data/submission_data.csv', index_col="time_stamp")

# 7. Setpoint가 포함된 column 제거
submission_df = submission_df.drop(columns=[col for col in submission_df.columns if 'Setpoint' in col])

# 8. Stage 1 및 Stage 2의 input 데이터 구성
stage1_submission_input_df = submission_df[stage1_input]
stage2_submission_input_df = submission_df[stage2_input]

# 9. NaN 값 처리 (각 feature의 평균값으로 대체)
imputer = SimpleImputer(strategy='mean')
stage1_input_df = pd.DataFrame(imputer.fit_transform(stage1_input_df), columns=stage1_input_df.columns)
stage2_input_df = pd.DataFrame(imputer.fit_transform(stage2_input_df), columns=stage2_input_df.columns)
stage1_submission_input_df = pd.DataFrame(imputer.transform(stage1_submission_input_df), columns=stage1_submission_input_df.columns)
stage2_submission_input_df = pd.DataFrame(imputer.transform(stage2_submission_input_df), columns=stage2_submission_input_df.columns)

# 10. XGBoost 모델 학습
# Stage 1 XGBoost 모델 학습
stage1_xgb_model = XGBRegressor()
stage1_xgb_model.fit(stage1_input_df, stage1_output_df)

# Stage 2 XGBoost 모델 학습
stage2_xgb_model = XGBRegressor()
stage2_xgb_model.fit(stage2_input_df, stage2_output_df)

# 11. Stage 1 및 Stage 2 예측
stage1_xgb_predictions = stage1_xgb_model.predict(stage1_submission_input_df)
stage2_xgb_predictions = stage2_xgb_model.predict(stage2_submission_input_df)

# 12. 예측값을 numpy 파일로 저장
np.save('xgb_submission1.npy', stage1_xgb_predictions)
np.save('xgb_submission2.npy', stage2_xgb_predictions)

print("XGBoost 모델로 학습 및 예측 완료")

In [ ]:
# 필수 라이브러리
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.impute import SimpleImputer

# 1. 전체 데이터셋 읽어들이기
raw_df = pd.read_csv('./data/continuous_factory_process.csv', index_col="time_stamp")

# 2. Setpoint가 포함된 column 제거
df = raw_df.drop(columns=[col for col in raw_df.columns if 'Setpoint' in col])

# 3. Stage 1, Stage 2의 output feature name을 정의
stage1_output = [col for col in df.columns if 'Stage1.Output' in col]
stage2_output = [col for col in df.columns if 'Stage2.Output' in col]
print(len(stage1_output), len(stage2_output))

# 4. Stage 1, Stage 2의 input feature name 정의
stage1_input = [col for col in df.columns if col not in stage1_output]
stage2_input = [col for col in df.columns if col not in stage2_output]

# 5. Stage 1, Stage 2 데이터셋 구성
stage1_input_df = df[stage1_input]
stage1_output_df = df[stage1_output]
stage2_input_df = df[stage2_input]
stage2_output_df = df[stage2_output]

# 6. test set(X_test) 데이터 읽어들이기
submission_df = pd.read_csv('./data/submission_data.csv', index_col="time_stamp")

# 7. Setpoint가 포함된 column 제거
submission_df = submission_df.drop(columns=[col for col in submission_df.columns if 'Setpoint' in col])

# 8. Stage 1 및 Stage 2의 input 데이터 구성
stage1_submission_input_df = submission_df[stage1_input]
stage2_submission_input_df = submission_df[stage2_input]

# 9. NaN 값 처리 (각 feature의 평균값으로 대체)
imputer = SimpleImputer(strategy='mean')
stage1_input_df = pd.DataFrame(imputer.fit_transform(stage1_input_df), columns=stage1_input_df.columns)
stage2_input_df = pd.DataFrame(imputer.fit_transform(stage2_input_df), columns=stage2_input_df.columns)
stage1_submission_input_df = pd.DataFrame(imputer.transform(stage1_submission_input_df), columns=stage1_submission_input_df.columns)
stage2_submission_input_df = pd.DataFrame(imputer.transform(stage2_submission_input_df), columns=stage2_submission_input_df.columns)

# 12. 모델 학습 (LightGBM 사용)
# Stage 1 LightGBM 모델 학습
stage1_lgb_model = LGBMRegressor()
stage1_lgb_model.fit(stage1_input_df, stage1_output_df)

# Stage 2 LightGBM 모델 학습
stage2_lgb_model = LGBMRegressor()
stage2_lgb_model.fit(stage2_input_df, stage2_output_df)

# 13. Stage 1 및 Stage 2 예측 (LightGBM)
stage1_lgb_predictions = stage1_lgb_model.predict(stage1_submission_input_df)
stage2_lgb_predictions = stage2_lgb_model.predict(stage2_submission_input_df)

np.save('lgb_submission1.npy', stage1_lgb_predictions)
np.save('lgb_submission2.npy', stage2_lgb_predictions)

print("XGBoost와 LightGBM 모델로 학습 및 예측 완료")